In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json
import os
import sys
sys.path.append("../lib")
from dataUtils import *
import cPickle as pickle
import cv2
from collections import Counter
from scipy.interpolate import UnivariateSpline
from scipy.stats import entropy 
from scipy.stats import chisquare
from scipy.stats import ttest_ind


In [2]:
popular_color_features = "../Final_features/vine_features_ordered.csv"
unpopular_color_features = "../Final_features/Unpopular2016_aesthetic_ordered.csv"
aesthetic_color_features = "../Logs/aesthetic_features_6.csv"
#aesthetic_csv = "../Final_features/total_aesthetic_thirds.pk"
aesthetic_csv = "../Final_features/Thirds_aesthetic_thirds.pk"
aesthetic_pop_temporal = "../Final_features/popTemporalSampledAesthetic.csv"
aesthetic_unpop_temporal = "../Final_features/unpopTemporalSampledAesthetic.csv"

instaAesthetic = "../Final_features/instaSamplesSorted.csv"

In [35]:
def readFeatureFile(color_features):
    with open(color_features) as g:
        featureLines = g.readlines()
    print len(featureLines)
    return featureLines

def readAesFeatuers(lines):
    features = []
    for line in lines:
        components = line.strip().split('|')
        features.append([float(x) for x in components[1:]])
    return features

def readFeatures(lines):
    ids = []
    features = []
    feats = []
    
    components = lines[0].strip().split(',')
    ID = components[0].strip().split('_')[0]
    
    for line in lines:
        components = line.strip().split('|')
        postId = components[0].strip().split('_')[0]
        if postId != ID:
            ids.append(postId)
            ID = postId
            #Take median of all frames in a vine for all the attributes
            #median = np.median(feats,0)
            median = feats[0]
            for i in range(len(median)):
                if np.isnan(median[i]):
                    median[i] = 0.0
            features.append(median)
            feats[:] = []
            feats.append([float(x) for x in components[1:]]) 
        else :
            feats.append([float(x) for x in components[1:]]) 
            
    return features, ids
    

def readFeaturesAsVector(lines):
    ids = []
    features = []
    feats = []
    
    components = lines[0].strip().split(',')
    ID = components[0].strip().split('_')[0]
    
    for line in lines:
        components = line.strip().split('|')
        postId = components[0].strip().split('_')[0]
        if postId != ID:
            ids.append(postId)
            ID = postId
            features.append(feats)
            feats[:] = []
            feats.append([float(x) for x in components[1:]]) 
        else :
            feats.append([float(x) for x in components[1:]]) 
            
    return features, ids

def readInstaFeaturesAsVector(lines):
    ids = []
    features = []
    feats = dict()
    
    components = lines[0].strip().split(',')
    ID = components[0].strip().split('+')[0]
    
    for line in lines:
        components = line.strip().split('|')
        postId = components[0].strip().split('+')[0]
        sequence = components[0].strip().split('+')[1].split('.')[0]
        if postId != ID:
            ids.append(postId)
            ID = postId
            f = []
            #print feats
            for i in range(1,len(feats.keys())+1):
                try:
                    f.append(feats[str(i)])
                except KeyError:
                    print len(feats.keys()) , i
            features.append(f)
            feats.clear()
            feats[sequence] = [float(x) for x in components[1:]]
            #feats.append([float(x) for x in components[1:]])
        else :
            feats[sequence] = [float(x) for x in components[1:]]
            #feats.append([float(x) for x in components[1:]]) 
            
    return features, ids

def getMaximumHistogram(featureVec , dimension ,function):
    classifierCoeefs = np.asarray([ 0.       ,   0.03343475,  0.0218128 ,  0.02341605 , 0.02355142,  0.02526042,
  0.02290093,  0.02206709,  0.02891414,  0.02192985,  0.01503583,  0.,
  0.02182189,  0.02630779 , 0.02273517,  0.02183237,  0.02532157,  0.02034298,
  0.01239432])
    
    maxClassifierCoefs = np.asarray([[ 0.       ,   0.03453842,  0.02570736,  0.02543318,  0.02516512,  0.02729427,
  0.02423508,  0.02827428,  0.02917423,  0.02597833,  0.01458388,  0.,
  0.02122234,  0.02544339,  0.02607218,  0.02100649,  0.02610868,  0.02172767,
  0.01354962]])
    max_hist = np.zeros(7)
    for video in featureVec:
        vec = [np.dot(maxClassifierCoefs,video[i][:]) for i in range(len(video))]
        maxArg = function(np.asarray(vec))
        max_hist[maxArg]+=1
    return [np.sum(max_hist[:2])/len(featureVec) , np.sum(max_hist[2:4])/len(featureVec) , np.sum(max_hist[4:6])/len(featureVec) ]

def getMaximumHistInsta(features, function):
    maxClassifierCoefs = np.asarray([[ 0.       ,   0.03453842,  0.02570736,  0.02543318,  0.02516512,  0.02729427,
    0.02423508,  0.02827428,  0.02917423,  0.02597833,  0.01458388,  0.,
    0.02122234,  0.02544339,  0.02607218,  0.02100649,  0.02610868,  0.02172767,
    0.01354962]])
    max_hist = np.zeros(3)
    for video in features:
        vec = [np.median(video[i][:]) for i in range(len(video))]
        maxArg = function(np.asarray(vec))
        thirds = (len(video)/3) + 1
        max_hist[int(maxArg/thirds)]+=1
    return max_hist
    

In [25]:
poptempLines = readFeatureFile(popular_color_features)



50187


In [26]:
unpoptempLines = readFeatureFile(aesthetic_unpop_temporal)

1210


In [27]:
instaLines = readFeatureFile(instaAesthetic)

3780


In [28]:
instaFeatureVec , instaPosts = readInstaFeaturesAsVector(instaLines)

In [29]:
popFeatureVec , popPosts = readFeaturesAsVector(poptempLines)

In [30]:
unpopFeatureVec , unpopPosts = readFeaturesAsVector(unpoptempLines)

In [31]:
len(instaFeatureVec)

199

In [32]:
maxPopVec = getMaximumHistogram(popFeatureVec,17 ,np.argmax)

maxunPopVec = getMaximumHistogram(unpopFeatureVec,17 ,np.argmax)

In [36]:
maxInstaVec = getMaximumHistInsta(instaFeatureVec , np.argmax)

/usr/local/lib/python2.7/dist-packages/numpy/lib/function_base.py:3569: RuntimeWarning: Invalid value encountered in median
  RuntimeWarning)


In [37]:
print maxPopVec
print maxunPopVec
print maxInstaVec

[0.0, 1.0, 0.0]
[0.0, 0.0, 1.0]
[ 89.  77.  33.]


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
ind = np.arange(1,4) 
width = 0.1
## the bars
rects1 = ax.bar(ind, maxPopVec, width,
                color='b')
#rects2 = ax.bar(ind+width, minPopVec, width,
#                color='g')
# rects3 = ax.bar(ind+2*width, maxunPopVec, width,
#                 color='r')
rects4 = ax.bar(ind+width, maxunPopVec, width,
                 color='purple')

# add some text for labels, title and axes ticks
ax.set_ylabel('Cumulative Aesthetic Impact',fontsize=15)
ax.set_xlabel('Which Third of the Video',fontsize=15)
plt.legend(['Max Cummulative aesthetic occurance - Popular' , 'Max Cummulative aesthetic occurance - Unpopular' ]
            , loc='upper right')
ax.set_xticks(ind )

plt.show()

In [ ]:
pop_features = readFeatureFile(popular_color_features)
unpop_features = readFeatureFile(unpopular_color_features)
aesthetic_features = readFeatureFile(aesthetic_color_features)

In [ ]:
popfeatures_matrix , popposts = readFeatures(pop_features)

In [ ]:
unpopfeatures_matrix , unpopposts = readFeatures(unpop_features)

In [ ]:
aes_feature = readAesFeatuers(aesthetic_features)

In [ ]:
print len(popposts) , len(unpopposts)

In [ ]:
totalPosts = np.asarray(popposts + unpopposts)
print totalPosts.shape
print totalPosts[1]

In [ ]:
totalFeats = np.asarray(popfeatures_matrix + unpopfeatures_matrix)
print totalFeats.shape
print totalFeats[15012]

In [ ]:
totalMap = np.append(totalPosts.reshape((-1,1)),totalFeats , axis=1)
print totalMap[15012]

In [ ]:
import pickle
f = open(aesthetic_csv , "w+")
pickle.dump(totalMap, f)
f.close()

In [ ]:
f = open(aesthetic_csv , "r+")
feats = pickle.load(f)
f.close()
print feats[104]

In [ ]:
unpop_aes_features = np.asarray(unpopfeatures_matrix)
aes_feats = np.asarray(aes_feature)
pop_aes_features = np.asarray(popfeatures_matrix)

print unpop_aes_features.shape , aes_feats.shape , pop_aes_features.shape

In [ ]:
samples_pop = np.random.choice(pop_aes_features.shape[0],1000)
samples_unpop = np.random.choice(unpop_aes_features.shape[0], 1000)
samples_aes = np.random.choice(aes_feats.shape[0], 1000)
sampledpop_aes_features = pop_aes_features[samples_pop,:]
sampledunpop_aes_features = unpop_aes_features[samples_unpop,:]
sampled_aes = aes_feats[samples_aes,:]

In [ ]:
print sampledpop_aes_features.shape , sampledunpop_aes_features.shape

In [ ]:
num = 7
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(sampledunpop_aes_features[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(sampled_aes[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' , 'Aesthetic sample images'])
plt.title('Rule of Thirds distribution for unpopular and popular videos', fontsize = 25)
plt.show()

print np.mean(sampledpop_aes_features[:,num]) , np.var(sampledpop_aes_features[:,num])
print np.mean(unpop_aes_features[:,num]) , np.var(unpop_aes_features[:,num])
print np.mean(sampled_aes[:,num]) , np.var(sampled_aes[:,num])

In [ ]:
num = 4
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(unpop_aes_features[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(sampled_aes[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' , 'Aesthetic Image dataset'])
plt.title('Modified Luo Simplicity distribution for unpopular and popular videos', fontsize = 25)
plt.show()
print np.mean(sampledpop_aes_features[:,num]) , np.var(sampledpop_aes_features[:,num])
print np.mean(unpop_aes_features[:,num]) , np.var(unpop_aes_features[:,num])
print np.mean(sampled_aes[:,num]) , np.var(sampled_aes[:,num])

In [ ]:
for i in range(sampledunpop_aes_features.shape[1]):
    print "Index %d" %i
    print "Aesthetic images: %0.3f , %0.3f , %0.3f "%( np.mean(aes_feats[:,i]), np.median(aes_feats[:,i]) , np.var(aes_feats[:,i]) )
    print "Popolar Vines: %0.3f , %0.3f , %0.3f "%( np.mean(sampledpop_aes_features[:,i]), np.median(sampledpop_aes_features[:,i]) , np.var(sampledpop_aes_features[:,i]) )
    print "Unpopular Vines: %0.3f , %0.3f , %0.3f "%( np.mean(sampledunpop_aes_features[:,i]), np.median(sampledunpop_aes_features[:,i]) , np.var(sampledunpop_aes_features[:,i]) )
    print "\n"
                                                                                                  

In [ ]:
print entropy(sampledpop_aes_features, sampled_aes, base=None)
print entropy(sampledunpop_aes_features, sampled_aes, base=None)


In [ ]:
num = 16
fig, ax = plt.subplots()
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0 )
plt.hist(unpop_aes_features[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(aes_feats[:,num], 1000, normed=1,histtype='step', cumulative=True, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' ,'Aesthetic Image dataset'])
plt.title('Sharp pixel proportion distribution for popular , unpopular videos Vs Top aesthetic images', fontsize = 25)
plt.grid()
plt.show()
print np.mean(sampledpop_aes_features[:,num]) , np.var(sampledpop_aes_features[:,num])
print np.mean(unpop_aes_features[:,num]) , np.var(unpop_aes_features[:,num])
print np.mean(sampled_aes[:,num]) , np.var(sampled_aes[:,num])

In [ ]:
print np.mean(sampledpop_aes_features[:,16]) , np.median(sampledpop_aes_features[:,16])
print np.mean(unpop_aes_features[:,16]) , np.median(unpop_aes_features[:,16])
print np.mean(aes_feats[:,16]) , np.median(aes_feats[:,16])

In [ ]:
fig, ax = plt.subplots()
num = 4
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 500, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(sampledunpop_aes_features[:,num], 500, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(aes_feats[:,num], 500, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' , 'Aesthetic sample images'])
plt.title('Rule of Thirds distribution for unpopular and popular videos', fontsize = 25)
plt.show()

In [ ]:
fig, ax = plt.subplots()
num = 3
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 100, normed=0,histtype='step', cumulative=False, linewidth = 1.0)
plt.hist(sampledunpop_aes_features[:,num], 100, normed=0,histtype='step', cumulative=False, linewidth = 1.0)
plt.hist(aes_feats[:,num], 100, normed=0,histtype='step', cumulative=False, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' , 'Aesthetic sample images'])
plt.title('Luo Simplicity distribution for different samples', fontsize = 25)
plt.show()

print np.mean(sampledpop_aes_features[:,num]) , np.median(sampledpop_aes_features[:,num])
print np.mean(unpop_aes_features[:,num]) , np.median(unpop_aes_features[:,num])
print np.mean(aes_feats[:,num]) , np.median(aes_feats[:,num])
t, p = ttest_ind( sampledpop_aes_features[:,num], sampled_aes[:,num])
print p
t, p = ttest_ind( sampledunpop_aes_features[:,num], sampled_aes[:,num])
print p


In [ ]:
fig, ax = plt.subplots()
num = 16
fig.set_size_inches(15, 10)
plt.hist(sampledpop_aes_features[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(sampledunpop_aes_features[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.hist(aes_feats[:,num], 1000, normed=0,histtype='step', cumulative=True, linewidth = 1.0)
plt.legend(['Popular Posts', 'Unpopular Posts' , 'Aesthetic sample images'])
plt.title('Luo Simplicity distribution for different samples', fontsize = 25)
plt.show()

print np.mean(sampledpop_aes_features[:,num]) , np.median(sampledpop_aes_features[:,num])
print np.mean(unpop_aes_features[:,num]) , np.median(unpop_aes_features[:,num])
print np.mean(aes_feats[:,num]) , np.median(aes_feats[:,num])